In [10]:
import boto3
import joblib
import pandas as pd
from prophet import Prophet

s3 = boto3.client('s3')

bucket_name = 'sleep-quality-forecast-bucket'
object_keys = [
                'models/prophet_AHI.pkl',
                'models/prophet_LEAK_95_PERCENTILE.pkl', 
                'models/prophet_MASK_SESSION_COUNT.pkl', 
                'models/prophet_SLEEP_SCORE.pkl', 
                'models/prophet_USAGE_HOURS.pkl'
              ]

In [15]:
results = {}

# Loop through each saved model in S3
for key in object_keys:
    # extract metric name
    metric_name = key.split('/')[-1].replace('prophet_', '').replace('.pkl', '')

    # download model locally
    local_path = f"/tmp/{metric_name}.pkl"
    s3.download_file(bucket_name, key, local_path)

    # load model
    model = joblib.load(local_path)

    # create dates 1 week in future
    future = model.make_future_dataframe(periods=7)
    forecast = model.predict(future)

    # get the last week
    next_week = forecast.tail(7)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

    # save results
    results[metric_name] = next_week

    # print
    print(f"\n7 Day forecast for {metric_name}:")
    print(next_week)
    print("-" * 50)


7 Day forecast for AHI:
            ds      yhat  yhat_lower  yhat_upper
852 2025-10-03  0.328244   -0.005713    0.695554
853 2025-10-04  0.369640    0.032557    0.724001
854 2025-10-05  0.359469    0.011204    0.701981
855 2025-10-06  0.369794    0.051672    0.704040
856 2025-10-07  0.371598    0.019797    0.707166
857 2025-10-08  0.338091   -0.001218    0.670484
858 2025-10-09  0.330492   -0.029819    0.643692
--------------------------------------------------

7 Day forecast for LEAK_95_PERCENTILE:
            ds      yhat  yhat_lower  yhat_upper
852 2025-10-03  5.263238   -0.038565   10.123402
853 2025-10-04  5.095033    0.232825   10.006537
854 2025-10-05  5.645448    0.831033   10.532345
855 2025-10-06  5.210778    0.140730    9.851218
856 2025-10-07  5.043177    0.231768   10.239111
857 2025-10-08  5.140124    0.061516    9.635305
858 2025-10-09  4.893194   -0.093396    9.795171
--------------------------------------------------

7 Day forecast for MASK_SESSION_COUNT:
         

In [17]:
# Save each forecast to S3 as CSV
for metric_name, df_forecast in results.items():
    csv_path = f"/tmp/{metric_name}_forecast.csv"
    df_forecast.to_csv(csv_path, index=False)
    s3.upload_file(csv_path, bucket_name, f"forecasts/{metric_name}_forecast.csv")
    print(f"Uploaded {metric_name}_forecast.csv to s3://{bucket_name}/forecasts/")

Uploaded AHI_forecast.csv to s3://sleep-quality-forecast-bucket/forecasts/
Uploaded LEAK_95_PERCENTILE_forecast.csv to s3://sleep-quality-forecast-bucket/forecasts/
Uploaded MASK_SESSION_COUNT_forecast.csv to s3://sleep-quality-forecast-bucket/forecasts/
Uploaded SLEEP_SCORE_forecast.csv to s3://sleep-quality-forecast-bucket/forecasts/
Uploaded USAGE_HOURS_forecast.csv to s3://sleep-quality-forecast-bucket/forecasts/
